# 2장 시계열 데이터의 발견 및 다루기

In [2]:
import pandas as pd

YearJoined = pd.read_csv('Ch02/data/year_joined.csv')

YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [3]:
emails = pd.read_csv('Ch02/data/emails.csv')

emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [4]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


In [6]:
# (max(emails[emails.user==998].week) - min(emails[emails.user==998].week)).days/7

In [7]:
max(emails[emails.user==998].week)

'2018-05-28 00:00:00'

In [8]:
import datetime
datetime.datetime.strptime(max(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S')

datetime.datetime(2018, 5, 28, 0, 0)

In [9]:
(datetime.datetime.strptime(max(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(min(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S')).days/7

25.0

In [10]:
emails[emails.user==998].shape

(24, 3)

In [11]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [12]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value = 0).reset_index()

all_email.columns=['week', 'user', 'emailsOpened']

In [13]:
all_email[all_email.user==998].sort_values('week')

,week,user,emailsOpened
14013,2015-02-09 00:00:00,998.0,0.0
42580,2015-02-16 00:00:00,998.0,0.0
3233,2015-02-23 00:00:00,998.0,0.0
1077,2015-03-02 00:00:00,998.0,0.0
68991,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
86778,2018-04-30 00:00:00,998.0,3.0
81927,2018-05-07 00:00:00,998.0,3.0
30183,2018-05-14 00:00:00,998.0,3.0
65757,2018-05-21 00:00:00,998.0,3.0


In [14]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()

cutoff_dates = cutoff_dates.reset_index()

In [15]:
for _, row in cutoff_dates.iterrows():
  member = row['user']
  start_date = row['min']
  end_date = row['max']

  all_email.drop(all_email[all_email.user == member][all_email.week < start_date].index, inplace=True)

  all_email.drop(all_email[all_email.user == member][all_email.week > end_date].index, inplace=True)

<ipython-input-15-8325b9330e44>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week < start_date].index, inplace=True)
<ipython-input-15-8325b9330e44>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week > end_date].index, inplace=True)
<ipython-input-15-8325b9330e44>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week < start_date].index, inplace=True)
<ipython-input-15-8325b9330e44>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week > end_date].index, inplace=True)
<ipython-input-15-8325b9330e44>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.

In [16]:
donations = pd.read_csv('Ch02/data/donations.csv')

In [18]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())

<ipython-input-18-a6d693572295>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())


In [30]:
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())

<ipython-input-30-f53e5220f04a>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())


In [22]:
agg_don = donations.groupby('user').apply(
    lambda df: df.amount.resample('W-MON').sum().loc[lambda x: x != 0]
)

<ipython-input-22-e0f3e960d83f>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_don = donations.groupby('user').apply(


In [35]:
agg_donations = pd.DataFrame(agg_don).reset_index()

In [48]:
agg_donations

,user,timestamp,amount
0,0.0,2015-03-30,25.0
1,0.0,2015-04-06,0.0
2,0.0,2015-04-13,0.0
3,0.0,2015-04-20,0.0
4,0.0,2015-04-27,0.0
...,...,...,...
32347,995.0,2017-09-11,0.0
32348,995.0,2017-09-18,0.0
32349,995.0,2017-09-25,0.0
32350,995.0,2017-10-02,1000.0


In [53]:
for i, j in all_email.groupby('user'):
  print(i, j)

1.0                       week  user  emailsOpened
0      2015-09-21 00:00:00   1.0           2.0
1078   2015-11-02 00:00:00   1.0           3.0
1617   2016-10-17 00:00:00   1.0           3.0
2156   2018-04-23 00:00:00   1.0           3.0
3773   2015-07-20 00:00:00   1.0           2.0
...                    ...   ...           ...
90552  2018-01-01 00:00:00   1.0           3.0
91091  2016-04-18 00:00:00   1.0           0.0
91630  2015-07-13 00:00:00   1.0           2.0
92169  2017-08-28 00:00:00   1.0           3.0
92708  2017-06-05 00:00:00   1.0           3.0

[153 rows x 3 columns]
3.0                       week  user  emailsOpened
2157   2018-04-23 00:00:00   3.0           2.0
54979  2018-04-09 00:00:00   3.0           1.0
63064  2018-03-26 00:00:00   3.0           2.0
79234  2018-04-02 00:00:00   3.0           1.0
83007  2018-03-19 00:00:00   3.0           3.0
83546  2018-03-12 00:00:00   3.0           3.0
88397  2018-04-16 00:00:00   3.0           2.0
90014  2018-03-05 00:00:00  

In [58]:
all_email

,week,user,emailsOpened
0,2015-09-21 00:00:00,1.0,2.0
13,2015-09-21 00:00:00,31.0,0.0
15,2015-09-21 00:00:00,34.0,2.0
16,2015-09-21 00:00:00,38.0,1.0
19,2015-09-21 00:00:00,42.0,3.0
...,...,...,...
93231,2017-06-05 00:00:00,959.0,0.0
93236,2017-06-05 00:00:00,970.0,2.0
93237,2017-06-05 00:00:00,973.0,3.0
93241,2017-06-05 00:00:00,987.0,3.0


In [ ]:
all_email['week'] = pd.to_datetime(all_email['week'])
all_email['week'] = all_email['week'].dt.to_period('W-MON').dt.start_time

agg_donations['timestamp'] = agg_donations['timestamp'].dt.to_period('W-MON').dt.start_time

In [85]:
merged_df = pd.DataFrame()

for member, member_email in all_email.groupby('user'):
  member_donations = agg_donations[agg_donations.user==member]

  member_donations.set_index('timestamp', inplace=True)

  member_email = all_email[all_email.user==member]
  print(member_email)
  member_email.week = pd.to_datetime(member_email.week)
  member_email.set_index('week', inplace=True)

  df = pd.merge(member_email, member_donations, how='left', left_index=True, right_index=True)
  print(df)

  df = df.fillna(0)

  df['member'] = df.user_x
  merged_df = pd.concat([merged_df, (df.reset_index()[['member', 'week', 'emailsOpened', 'amount']])])

            week  user  emailsOpened
0     2015-09-15   1.0           2.0
1078  2015-10-27   1.0           3.0
1617  2016-10-11   1.0           3.0
2156  2018-04-17   1.0           3.0
3773  2015-07-14   1.0           2.0
...          ...   ...           ...
90552 2017-12-26   1.0           3.0
91091 2016-04-12   1.0           0.0
91630 2015-07-07   1.0           2.0
92169 2017-08-22   1.0           3.0
92708 2017-05-30   1.0           3.0

[153 rows x 3 columns]
            user_x  emailsOpened  user_y  amount
week                                            
2015-09-15     1.0           2.0     NaN     NaN
2015-10-27     1.0           3.0     NaN     NaN
2016-10-11     1.0           3.0     1.0     0.0
2018-04-17     1.0           3.0     NaN     NaN
2015-07-14     1.0           2.0     NaN     NaN
...            ...           ...     ...     ...
2017-12-26     1.0           3.0     NaN     NaN
2016-04-12     1.0           0.0     NaN     NaN
2015-07-07     1.0           2.0     NaN  

<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            week  user  emailsOpened
2188  2018-04-17  69.0           3.0
5961  2017-09-19  69.0           2.0
9734  2018-02-06  69.0           3.0
10273 2017-09-05  69.0           3.0
12429 2017-11-28  69.0           3.0
...          ...   ...           ...
89506 2017-09-12  69.0           3.0
90045 2018-02-27  69.0           3.0
90584 2017-12-26  69.0           3.0
92201 2017-08-22  69.0           3.0
92740 2017-05-30  69.0           3.0

[68 rows x 3 columns]
            user_x  emailsOpened  user_y  amount
week                                            
2018-04-17    69.0           3.0     NaN     NaN
2017-09-19    69.0           2.0    69.0     0.0
2018-02-06    69.0           3.0     NaN     NaN
2017-09-05    69.0           3.0    69.0     0.0
2017-11-28    69.0           3.0     NaN     NaN
...            ...           ...     ...     ...
2017-09-12    69.0           3.0    69.0     0.0
2018-02-27    69.0           3.0     NaN     NaN
2017-12-26    69.0           3.0     NaN   

<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            week   user  emailsOpened
1145  2015-10-27  129.0           2.0
11925 2016-06-21  129.0           1.0
18932 2015-11-17  129.0           0.0
20549 2016-04-26  129.0           0.0
22166 2015-10-20  129.0           3.0
23244 2016-01-26  129.0           0.0
24322 2016-03-15  129.0           0.0
30790 2016-03-22  129.0           0.0
33485 2016-02-23  129.0           0.0
35641 2016-03-29  129.0           1.0
38875 2016-05-17  129.0           0.0
41031 2016-02-09  129.0           0.0
44265 2016-01-12  129.0           0.0
44804 2015-11-03  129.0           2.0
45343 2015-12-29  129.0           0.0
46421 2016-05-10  129.0           0.0
51811 2016-04-19  129.0           0.0
53967 2016-05-24  129.0           0.0
61513 2016-03-01  129.0           0.0
62052 2015-12-15  129.0           0.0
65825 2016-05-03  129.0           0.0
66364 2016-06-14  129.0           0.0
70676 2015-11-10  129.0           2.0
71215 2016-02-02  129.0           0.0
71754 2016-04-05  129.0           1.0
72293 2016-0

...

<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            week   user  emailsOpened
1026  2015-02-24  893.0           0.0
3182  2015-02-17  893.0           2.0
13962 2015-02-03  893.0           1.0
18813 2015-03-10  893.0           2.0
42529 2015-02-10  893.0           0.0
68940 2015-03-03  893.0           0.0
            user_x  emailsOpened  user_y  amount
week                                            
2015-02-24   893.0           0.0   893.0     0.0
2015-02-17   893.0           2.0   893.0     0.0
2015-02-03   893.0           1.0     NaN     NaN
2015-03-10   893.0           2.0   893.0     0.0
2015-02-10   893.0           0.0   893.0    50.0
2015-03-03   893.0           0.0   893.0     0.0
            week   user  emailsOpened
6417  2017-09-19  895.0           1.0
10190 2018-02-06  895.0           0.0
10729 2017-09-05  895.0           0.0
12885 2017-11-28  895.0           0.0
13424 2017-10-10  895.0           0.0
15580 2017-09-26  895.0           0.0
21509 2017-08-15  895.0           0.0
25282 2018-01-02  895.0           0.0


<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            week   user  emailsOpened
2676  2018-04-17  955.0           3.0
6449  2017-09-19  955.0           1.0
10222 2018-02-06  955.0           3.0
10761 2017-09-05  955.0           0.0
12917 2017-11-28  955.0           0.0
...          ...    ...           ...
89994 2017-09-12  955.0           0.0
90533 2018-02-27  955.0           3.0
91072 2017-12-26  955.0           3.0
92689 2017-08-22  955.0           0.0
93228 2017-05-30  955.0           0.0

[67 rows x 3 columns]
            user_x  emailsOpened  user_y  amount
week                                            
2018-04-17   955.0           3.0   955.0     0.0
2017-09-19   955.0           1.0     NaN     NaN
2018-02-06   955.0           3.0   955.0     0.0
2017-09-05   955.0           0.0     NaN     NaN
2017-11-28   955.0           0.0   955.0     0.0
...            ...           ...     ...     ...
2017-09-12   955.0           0.0     NaN     NaN
2018-02-27   955.0           3.0   955.0     0.0
2017-12-26   955.0           3.

<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-85-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [88]:
merged_df.loc[lambda x: x.amount != 0]

,member,week,emailsOpened,amount
14,1.0,2017-01-17,3.0,75.0
71,1.0,2016-05-10,3.0,50.0
104,1.0,2016-05-03,3.0,50.0
1,5.0,2017-09-19,1.0,25.0
24,5.0,2017-07-25,2.0,50.0
...,...,...,...,...
76,987.0,2018-01-16,3.0,100.0
12,993.0,2017-10-31,3.0,50.0
15,995.0,2017-09-26,3.0,1000.0
16,995.0,2016-09-13,3.0,1000.0


In [90]:
merged_df = merged_df.sort_values(by='week', ascending=True)

In [91]:
df = merged_df[merged_df.member == 998]

df['target'] = df.amount.shift(1)
df = df.fillna(0)
df

<ipython-input-91-a2aec95fca91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df.amount.shift(1)


,member,week,emailsOpened,amount,target
2,998.0,2017-11-28,1.0,0.0,0.0
6,998.0,2017-12-05,3.0,0.0,0.0
14,998.0,2017-12-12,3.0,0.0,0.0
10,998.0,2017-12-19,0.0,0.0,0.0
25,998.0,2017-12-26,3.0,0.0,0.0
3,998.0,2018-01-02,3.0,50.0,0.0
7,998.0,2018-01-09,2.0,0.0,50.0
16,998.0,2018-01-16,3.0,0.0,0.0
8,998.0,2018-01-23,2.0,0.0,0.0
18,998.0,2018-01-30,3.0,0.0,0.0
